데이터를 전처리할 것.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
## Data import
repress = pd.read_table("/content/drive/MyDrive/3. Random forest/sorted.activeEnhancer.EP300_1.both0.50.intersect.wMergedHistones.gencode.v34lift37.annotation.bed",header=None)
active1 = pd.read_table("/content/drive/MyDrive/3. Random forest/sorted.activeEnhancer.EP300_1.both0.50.intersect.wMergedHistones.gencode.v34lift37.annotation.bed",header=None)
active2 = pd.read_table("/content/drive/MyDrive/3. Random forest/sorted.activeEnhancer.EP300_2.both0.50.intersect.wMergedHistones.gencode.v34lift37.annotation.bed",header=None)

In [ ]:
## concatenate active1 and active2
# active file 두개를 합침.
active = pd.concat((active1, active2))

In [ ]:
active

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,chr1,1217050,1217426,.,866,.,26.36167,-1.0,0.97528,188,H3K4me3_2.bed,chr1,1217008,1217760,Peak_29792,35,.,3.81197,7.83255,5.76431,170,chr1,1215816,1227409,ENSG00000162572.21_5,SCNN1D,protein_coding,HAVANA,full_contig,0
1,chr1,3585407,3585783,.,650,.,24.96101,-1.0,0.96919,188,H3K4me3_1.bed,chr1,3585259,3585626,Peak_25316,41,.,7.62992,16.25384,14.17348,114,chr1,3569080,3652765,ENSG00000078900.15_8,TP73,protein_coding,HAVANA,full_contig,0
2,chr1,6264932,6265308,.,676,.,23.20480,-1.0,0.94664,188,H3K4me1.bed,chr1,6264673,6265299,Peak_77517,60,.,5.51486,15.63096,13.90281,244,chr1,6245080,6269449,ENSG00000116251.11_7,RPL22,protein_coding,HAVANA,full_contig,0
3,chr1,6264932,6265308,.,676,.,23.20480,-1.0,0.94664,188,H3K4me1.bed,chr1,6264673,6265299,Peak_77517,60,.,5.51486,15.63096,13.90281,244,chr1,6264900,6265840,ENSG00000226944.1_8,AL031847.1,lncRNA,HAVANA,full_contig,0
4,chr1,6264932,6265308,.,676,.,23.20480,-1.0,0.94664,188,H3K4me3_2.bed,chr1,6264806,6265350,Peak_17989,80,.,7.04172,18.39370,16.10255,183,chr1,6245080,6269449,ENSG00000116251.11_7,RPL22,protein_coding,HAVANA,full_contig,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8161,chrX,153276008,153276332,.,562,.,5.82131,-1.0,-0.01110,162,H3K9ac.bed,chrX,153276032,153276263,Peak_44249,16,.,4.33856,5.91645,3.94625,67,chrX,153275957,153285353,ENSG00000184216.14_10,IRAK1,protein_coding,HAVANA,full_contig,0
8162,chrX,153305894,153306218,.,620,.,19.21680,-1.0,2.16978,162,H3K79me2.bed,chrX,153306035,153306283,Peak_78273,46,.,4.90406,6.80165,4.88641,129,chrX,153287024,153402578,ENSG00000169057.23_9,MECP2,protein_coding,HAVANA,partial,0
8163,chrX,153587219,153587543,.,554,.,20.32860,-1.0,2.34613,162,H3K36me3.bed,chrX,153587090,153587429,Peak_73274,141,.,4.41572,4.79236,2.01952,76,chrX,153576892,153603006,ENSG00000196924.18_8,FLNA,protein_coding,HAVANA,full_contig,0
8164,chrX,153587219,153587543,.,554,.,20.32860,-1.0,2.34613,162,H3K4me2.bed,chrX,153587098,153587402,Peak_111663,15,.,3.32335,4.31654,2.72254,114,chrX,153576892,153603006,ENSG00000196924.18_8,FLNA,protein_coding,HAVANA,full_contig,0


In [ ]:
repress.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,chr1,1217050,1217426,.,866,.,26.36167,-1.0,0.97528,188,H3K4me3_2.bed,chr1,1217008,1217760,Peak_29792,35,.,3.81197,7.83255,5.76431,170,chr1,1215816,1227409,ENSG00000162572.21_5,SCNN1D,protein_coding,HAVANA,full_contig,0
1,chr1,3585407,3585783,.,650,.,24.96101,-1.0,0.96919,188,H3K4me3_1.bed,chr1,3585259,3585626,Peak_25316,41,.,7.62992,16.25384,14.17348,114,chr1,3569080,3652765,ENSG00000078900.15_8,TP73,protein_coding,HAVANA,full_contig,0
2,chr1,6264932,6265308,.,676,.,23.20480,-1.0,0.94664,188,H3K4me1.bed,chr1,6264673,6265299,Peak_77517,60,.,5.51486,15.63096,13.90281,244,chr1,6245080,6269449,ENSG00000116251.11_7,RPL22,protein_coding,HAVANA,full_contig,0
3,chr1,6264932,6265308,.,676,.,23.20480,-1.0,0.94664,188,H3K4me1.bed,chr1,6264673,6265299,Peak_77517,60,.,5.51486,15.63096,13.90281,244,chr1,6264900,6265840,ENSG00000226944.1_8,AL031847.1,lncRNA,HAVANA,full_contig,0
4,chr1,6264932,6265308,.,676,.,23.20480,-1.0,0.94664,188,H3K4me3_2.bed,chr1,6264806,6265350,Peak_17989,80,.,7.04172,18.39370,16.10255,183,chr1,6245080,6269449,ENSG00000116251.11_7,RPL22,protein_coding,HAVANA,full_contig,0


bed 전처리는 뭐.. 명령어를 통해서 할 수 있다.... 더라..
모양을 보니 active의 같음. chr1의 어디에 있다.

## column 정보

### represssor or active info
    "chrom","chromStart","chromEnd","string name","score","strand","signalValue","pValue","qValue","peak",  
### Histone info
    "Histone","Histone chrom","HchromStart","HchromEnd","Hstring name","Hscore","Hstrand","HsignalValue","HpValue","HqValue","Hpeak",
### genecode info
    "seqname", "start","end","gene_id","gene_name","gene_type","source","remap_status","distance with gene"

In [ ]:
## 필요한 정보 : "Histone", "HsignalValue", "seqname", "start","end", "gene_name", "distance with gene"

In [ ]:
## repressive enhancer
ezh2 = repress.loc[:,[10,17,21,22,23,25,29]] # 필요한 칼럼들만을 추출
ezh2.columns = ["Histone", "HsignalValue", "chr","start","end","gene_name", "distance with gene"] # 이름을 붙임
ezh2

,Histone,HsignalValue,chr,start,end,gene_name,distance with gene
0,H3K4me3_2.bed,3.81197,chr1,1215816,1227409,SCNN1D,0
1,H3K4me3_1.bed,7.62992,chr1,3569080,3652765,TP73,0
2,H3K4me1.bed,5.51486,chr1,6245080,6269449,RPL22,0
3,H3K4me1.bed,5.51486,chr1,6264900,6265840,AL031847.1,0
4,H3K4me3_2.bed,7.04172,chr1,6245080,6269449,RPL22,0
...,...,...,...,...,...,...,...
2170,H3K9ac.bed,4.33856,chrX,40374147,40374760,AC092473.1,1316
2171,H3K4me2.bed,4.81479,chrX,73160270,73513409,FTX,2224
2172,H3K4me3_3.bed,20.58983,chrX,117973502,118053632,LINC01285,0
2173,H3K9ac.bed,12.58181,chrX,117973502,118053632,LINC01285,0


In [ ]:
## active enhancer
ep300 = active.loc[:,[10,17,21,22,23,25,29]]  # 필요한 칼럼들만을 추출
ep300.columns =  ["Histone", "HsignalValue", "chr","start","end","gene_name", "distance with gene"] # 이름을 붙임
ep300

,Histone,HsignalValue,chr,start,end,gene_name,distance with gene
0,H3K4me3_2.bed,3.81197,chr1,1215816,1227409,SCNN1D,0
1,H3K4me3_1.bed,7.62992,chr1,3569080,3652765,TP73,0
2,H3K4me1.bed,5.51486,chr1,6245080,6269449,RPL22,0
3,H3K4me1.bed,5.51486,chr1,6264900,6265840,AL031847.1,0
4,H3K4me3_2.bed,7.04172,chr1,6245080,6269449,RPL22,0
...,...,...,...,...,...,...,...
8161,H3K9ac.bed,4.33856,chrX,153275957,153285353,IRAK1,0
8162,H3K79me2.bed,4.90406,chrX,153287024,153402578,MECP2,0
8163,H3K36me3.bed,4.41572,chrX,153576892,153603006,FLNA,0
8164,H3K4me2.bed,3.32335,chrX,153576892,153603006,FLNA,0


In [ ]:
# 필요한 두 가지 작엄.

# 1) chrom + chromStart + chromEnd -> Pos (chr:####-####) 몇번 chr인지, 몇부터 몇번째 위치까지인지를 다음 형태로 만들어줄 것. 
# 2) Histone.bed -> Histone

# 함수로 만들자.
def reshape_df(df): # df를 받아서
  df['Pos'] = 'temp' # 맨 끝에 Pos라는 열을 추가

  for i in range(len(df)) : # 행의 개수만큼
    df['Pos'][i] = '{} : {}-{}'.format(df["chr"][i], df["start"][i], df["end"][i]) # (chr:####-####) 형태로 변형해줌.
    df['Histone'][i] = df['Histone'][i].split('.')[0].split('_')[0]   # .을 기준으로 앞뒤로 쪼갠 후, 앞의 것으로 갱신 > .bed를 떼어냄 > 언더바 기준으로 앞뒤로 나눈 후 앞의 걸 선택 (_ 뒤를 떼어냄.)

  result = df.reindex(columns = ['Pos', 'gene_name', 'distance with gene', 'Histone', 'HsignalValue']) # 결과를 반환 시 인덱스를 재배열함.

  return result

In [ ]:
# ep300 데이터의 총 행 개수가 10341개 있는데, index가 8000 언저리에서 끝나므로 index를 맞춰줌.
ep300 = ep300.reset_index(drop=True)
ep300 

,Histone,HsignalValue,chr,start,end,gene_name,distance with gene
0,H3K4me3_2.bed,3.81197,chr1,1215816,1227409,SCNN1D,0
1,H3K4me3_1.bed,7.62992,chr1,3569080,3652765,TP73,0
2,H3K4me1.bed,5.51486,chr1,6245080,6269449,RPL22,0
3,H3K4me1.bed,5.51486,chr1,6264900,6265840,AL031847.1,0
4,H3K4me3_2.bed,7.04172,chr1,6245080,6269449,RPL22,0
...,...,...,...,...,...,...,...
10336,H3K9ac.bed,4.33856,chrX,153275957,153285353,IRAK1,0
10337,H3K79me2.bed,4.90406,chrX,153287024,153402578,MECP2,0
10338,H3K36me3.bed,4.41572,chrX,153576892,153603006,FLNA,0
10339,H3K4me2.bed,3.32335,chrX,153576892,153603006,FLNA,0


In [ ]:
EZH2 = reshape_df(ezh2)
EP300 = reshape_df(ep300)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
EZH2 # 결과 확인.

,Pos,gene_name,distance with gene,Histone,HsignalValue
0,chr1 : 1215816-1227409,SCNN1D,0,H3K4me3,3.81197
1,chr1 : 3569080-3652765,TP73,0,H3K4me3,7.62992
2,chr1 : 6245080-6269449,RPL22,0,H3K4me1,5.51486
3,chr1 : 6264900-6265840,AL031847.1,0,H3K4me1,5.51486
4,chr1 : 6245080-6269449,RPL22,0,H3K4me3,7.04172
...,...,...,...,...,...
2170,chrX : 40374147-40374760,AC092473.1,1316,H3K9ac,4.33856
2171,chrX : 73160270-73513409,FTX,2224,H3K4me2,4.81479
2172,chrX : 117973502-118053632,LINC01285,0,H3K4me3,20.58983
2173,chrX : 117973502-118053632,LINC01285,0,H3K9ac,12.58181


In [ ]:
EP300 # 결과 확인.

,Pos,gene_name,distance with gene,Histone,HsignalValue
0,chr1 : 1215816-1227409,SCNN1D,0,H3K4me3,3.81197
1,chr1 : 3569080-3652765,TP73,0,H3K4me3,7.62992
2,chr1 : 6245080-6269449,RPL22,0,H3K4me1,5.51486
3,chr1 : 6264900-6265840,AL031847.1,0,H3K4me1,5.51486
4,chr1 : 6245080-6269449,RPL22,0,H3K4me3,7.04172
...,...,...,...,...,...
10336,chrX : 153275957-153285353,IRAK1,0,H3K9ac,4.33856
10337,chrX : 153287024-153402578,MECP2,0,H3K79me2,4.90406
10338,chrX : 153576892-153603006,FLNA,0,H3K36me3,4.41572
10339,chrX : 153576892-153603006,FLNA,0,H3K4me2,3.32335


In [ ]:
# 인핸서 종류가 무엇인지를 추가.
EP300['enhancer'] = 'EP300'
EZH2['enhancer'] = 'EZH2'

In [ ]:
enhancers = pd.concat([EZH2, EP300])
enhancers = enhancers.reset_index(drop=True) # 인덱스 개수가 맞도록 재조정.
enhancers

,Pos,gene_name,distance with gene,Histone,HsignalValue,enhancer
0,chr1 : 1215816-1227409,SCNN1D,0,H3K4me3,3.81197,EZH2
1,chr1 : 3569080-3652765,TP73,0,H3K4me3,7.62992,EZH2
2,chr1 : 6245080-6269449,RPL22,0,H3K4me1,5.51486,EZH2
3,chr1 : 6264900-6265840,AL031847.1,0,H3K4me1,5.51486,EZH2
4,chr1 : 6245080-6269449,RPL22,0,H3K4me3,7.04172,EZH2
...,...,...,...,...,...,...
12511,chrX : 153275957-153285353,IRAK1,0,H3K9ac,4.33856,EP300
12512,chrX : 153287024-153402578,MECP2,0,H3K79me2,4.90406,EP300
12513,chrX : 153576892-153603006,FLNA,0,H3K36me3,4.41572,EP300
12514,chrX : 153576892-153603006,FLNA,0,H3K4me2,3.32335,EP300


In [ ]:
## 동일한 유전자에서 같은 histone이 발견되면 distance 적은 걸로, 같으면 HsignalValue가 큰 값을 남기기. 같으면 둘중 하나 없애 list에 넣어서 drop(list)

enhancers[enhancers["gene_name"] == 'RPL22'] # 이거 보면 같은 gene 안에서도 histone이 다른 게 붙어있을 수도 있고 같기도 하다.
# 이걸 어떻게 처리할거냐면.. 같은 histone code면 distance가 적은 걸로 선택하고, 그것마저 같으면 HsignalValue가 큰 것을 남길 것임. (시그널이 크게 나오는걸로)


,Pos,gene_name,distance with gene,Histone,HsignalValue,enhancer
2,chr1 : 6245080-6269449,RPL22,0,H3K4me1,5.51486,EZH2
4,chr1 : 6245080-6269449,RPL22,0,H3K4me3,7.04172,EZH2
6,chr1 : 6245080-6269449,RPL22,0,H3K4me3,8.17219,EZH2
8,chr1 : 6245080-6269449,RPL22,0,H3K9ac,20.52876,EZH2
2177,chr1 : 6245080-6269449,RPL22,0,H3K4me1,5.51486,EP300
2179,chr1 : 6245080-6269449,RPL22,0,H3K4me3,7.04172,EP300
2181,chr1 : 6245080-6269449,RPL22,0,H3K4me3,8.17219,EP300
2183,chr1 : 6245080-6269449,RPL22,0,H3K9ac,20.52876,EP300


In [ ]:
# rm_list = [] # 임시 리스트를 만들고

# for gene in enhancers['gene_name'].unique() : # 고유 값들에 대해서
  
#   temp_df = enhancers[enhancers['gene_name'] == gene] # 해당 gene들을 임시로 temp_df에 넘겨서 하나를 만들고

#   for i in list(temp_df.index) :
#     for j in list(temp_df.index) :

#       if (i < j) & (temp_df.loc[i, "Histone"] == temp_df.loc[j, "Histone"]) & (temp_df.loc[i, "enhancer"]==temp_df.loc[j, "enhancer"]) : # histone과 enhancer가 같은가?
        
#         if temp_df.loc[i, 'distance with gene'] < temp_df.loc[j, 'distance with gene'] : # i의 distance가 적으면 i행을 넣어줌
#           rm_list.append(i) 
        
#         elif temp_df.loc[i, 'distance with gene'] > temp_df.loc[j, 'distance with gene'] : # j가 작으면 j를 넣어주고
#           rm_list.append(j)
        
#         elif temp_df.loc[i, 'distance with gene'] == temp_df.loc[j, 'distance with gene'] : # 두개의 거리가 같으면
          
#           if temp_df.loc[i, 'HsignalValue'] <= temp_df.loc[j, 'HsignalValue'] : # HsignalValue가 큰 것을 남긴다.
#             rm_list.append(j)
#           else :
#             rm_list.append(i)


rm_list = []
for gene in enhancers['gene_name'].unique():
    tmp_df = enhancers[enhancers['gene_name']==gene]
    for idx in list(tmp_df.index):
        for i in list(tmp_df.index):
            if (idx < i) & (tmp_df.loc[idx, "Histone"]==tmp_df.loc[i, "Histone"]) & (tmp_df.loc[idx,'enhancer']==tmp_df.loc[i,'enhancer']):
                if tmp_df.loc[idx, 'distance with gene'] < tmp_df.loc[i,'distance with gene']:
                    rm_list.append(i)
                elif tmp_df.loc[idx, 'distance with gene'] > tmp_df.loc[i,'distance with gene']:
                    rm_list.append(idx)
                elif tmp_df.loc[idx, 'distance with gene'] == tmp_df.loc[i,'distance with gene']:
                    if tmp_df.loc[idx, 'HsignalValue'] < tmp_df.loc[i, 'HsignalValue']:
                        rm_list.append(idx)
                    elif tmp_df.loc[idx, 'HsignalValue'] >= tmp_df.loc[i, 'HsignalValue']:
                        rm_list.append(i)

rm_list = list(set(rm_list)) # 
len(rm_list)

0

In [ ]:
enhancers = enhancers.drop(rm_list) # 해당 index에 해당하는 것을 골라서 버림.
enhancers = enhancers.reset_index(drop=True) # 인덱스 번호를 맞춰줌.
enhancers

,Pos,gene_name,distance with gene,Histone,HsignalValue,enhancer,value
0,chr1 : 1215816-1227409,SCNN1D,0,H3K4me3,3.81197,EZH2,1
1,chr1 : 3569080-3652765,TP73,0,H3K4me3,7.62992,EZH2,1
2,chr1 : 6245080-6269449,RPL22,0,H3K4me1,5.51486,EZH2,1
3,chr1 : 6264900-6265840,AL031847.1,0,H3K4me1,5.51486,EZH2,1
4,chr1 : 6245080-6269449,RPL22,0,H3K4me3,7.04172,EZH2,1
...,...,...,...,...,...,...,...
9803,chrX : 153275957-153285353,IRAK1,0,H3K9ac,4.33856,EP300,1
9804,chrX : 153287024-153402578,MECP2,0,H3K79me2,4.90406,EP300,1
9805,chrX : 153576892-153603006,FLNA,0,H3K36me3,4.41572,EP300,1
9806,chrX : 153576892-153603006,FLNA,0,H3K4me2,3.32335,EP300,1


In [ ]:
# reshape dataframe
pivotdf1 = pd.pivot_table(data = enhancers, index = ['Pos', 'gene_name'], columns = ['Histone'], fill_value=0, values= ['HsignalValue'])
pivotdf1

HsignalValue           ...                 
Histone                                   H3K27me3 H3K36me3  ... H3K9me3 H4K20me1
Pos                        gene_name                         ...                 
chr1 : 100111669-100160097 PALMD           3.73991      0.0  ...     0.0      0.0
chr1 : 100169526-100232191 FRRS1           0.00000      0.0  ...     0.0      0.0
chr1 : 100315917-100389579 AGL             0.00000      0.0  ...     0.0      0.0
chr1 : 100731763-100758325 RTCA            0.00000      0.0  ...     0.0      0.0
chr1 : 100810557-100985833 CDC14A          0.00000      0.0  ...     0.0      0.0
...                                            ...      ...  ...     ...      ...
chrX : 9217960-9243246     AC003684.1      0.00000      0.0  ...     0.0      0.0
chrX : 9983294-10112514    WWC3            0.00000      0.0  ...     0.0      0.0
chrX : 99899192-99930785   SRPX2           0.00000      0.0  ...     0.0      0.0
chrX : 9992882-10006694    WWC3-AS1        0.00000      0.0  ...     0.0      0.0
chrY : 16634453-16957530   NLGN4Y          0.00000      0.0  ...     0.0      0.0

[5492 rows x 9 columns]

In [ ]:
enhancers['value'] = 1
pivotdf2 = pd.pivot_table(data = enhancers, index = ['Pos', 'gene_name'], columns = ['enhancer'], fill_value=0, values= ['value'])
pivotdf2

value     
enhancer                              EP300 EZH2
Pos                        gene_name            
chr1 : 100111669-100160097 PALMD          1    0
chr1 : 100169526-100232191 FRRS1          1    0
chr1 : 100315917-100389579 AGL            1    0
chr1 : 100731763-100758325 RTCA           1    0
chr1 : 100810557-100985833 CDC14A         1    1
...                                     ...  ...
chrX : 9217960-9243246     AC003684.1     1    0
chrX : 9983294-10112514    WWC3           1    0
chrX : 99899192-99930785   SRPX2          1    0
chrX : 9992882-10006694    WWC3-AS1       1    0
chrY : 16634453-16957530   NLGN4Y         1    0

[5492 rows x 2 columns]

In [ ]:
mergedf = pivotdf1.merge(pivotdf2, left_index=True, right_index=True)
mergedf

HsignalValue           ... value     
Histone                                   H3K27me3 H3K36me3  ... EP300 EZH2
Pos                        gene_name                         ...           
chr1 : 100111669-100160097 PALMD           3.73991      0.0  ...     1    0
chr1 : 100169526-100232191 FRRS1           0.00000      0.0  ...     1    0
chr1 : 100315917-100389579 AGL             0.00000      0.0  ...     1    0
chr1 : 100731763-100758325 RTCA            0.00000      0.0  ...     1    0
chr1 : 100810557-100985833 CDC14A          0.00000      0.0  ...     1    1
...                                            ...      ...  ...   ...  ...
chrX : 9217960-9243246     AC003684.1      0.00000      0.0  ...     1    0
chrX : 9983294-10112514    WWC3            0.00000      0.0  ...     1    0
chrX : 99899192-99930785   SRPX2           0.00000      0.0  ...     1    0
chrX : 9992882-10006694    WWC3-AS1        0.00000      0.0  ...     1    0
chrY : 16634453-16957530   NLGN4Y          0.00000      0.0  ...     1    0

[5492 rows x 11 columns]

In [ ]:
## EP300과 EZH2 가 둘다 1이거나 둘다 0인 데이터 삭제

mergedf[ mergedf['EP300'] != mergedf['EZH2'] ] # 두개가 다른 것들만 살린다.
mergedf 

KeyError: ignored

In [ ]:
finaldf = mergedf.iloc[:,:-1] # 
finaldf

HsignalValue           ...          value
Histone                                   H3K27me3 H3K36me3  ... H4K20me1 EP300
Pos                        gene_name                         ...               
chr1 : 100111669-100160097 PALMD           3.73991      0.0  ...      0.0     1
chr1 : 100169526-100232191 FRRS1           0.00000      0.0  ...      0.0     1
chr1 : 100315917-100389579 AGL             0.00000      0.0  ...      0.0     1
chr1 : 100731763-100758325 RTCA            0.00000      0.0  ...      0.0     1
chr1 : 100810557-100985833 CDC14A          0.00000      0.0  ...      0.0     1
...                                            ...      ...  ...      ...   ...
chrX : 9217960-9243246     AC003684.1      0.00000      0.0  ...      0.0     1
chrX : 9983294-10112514    WWC3            0.00000      0.0  ...      0.0     1
chrX : 99899192-99930785   SRPX2           0.00000      0.0  ...      0.0     1
chrX : 9992882-10006694    WWC3-AS1        0.00000      0.0  ...      0.0     1
chrY : 16634453-16957530   NLGN4Y          0.00000      0.0  ...      0.0     1

[5492 rows x 10 columns]

In [ ]:
finaldf.columns.values[-1] = 'enhancer'
finaldf

HsignalValue           ...          value
Histone                                   H3K27me3 H3K36me3  ... H4K20me1 EP300
Pos                        gene_name                         ...               
chr1 : 100111669-100160097 PALMD           3.73991      0.0  ...      0.0     1
chr1 : 100169526-100232191 FRRS1           0.00000      0.0  ...      0.0     1
chr1 : 100315917-100389579 AGL             0.00000      0.0  ...      0.0     1
chr1 : 100731763-100758325 RTCA            0.00000      0.0  ...      0.0     1
chr1 : 100810557-100985833 CDC14A          0.00000      0.0  ...      0.0     1
...                                            ...      ...  ...      ...   ...
chrX : 9217960-9243246     AC003684.1      0.00000      0.0  ...      0.0     1
chrX : 9983294-10112514    WWC3            0.00000      0.0  ...      0.0     1
chrX : 99899192-99930785   SRPX2           0.00000      0.0  ...      0.0     1
chrX : 9992882-10006694    WWC3-AS1        0.00000      0.0  ...      0.0     1
chrY : 16634453-16957530   NLGN4Y          0.00000      0.0  ...      0.0     1

[5492 rows x 10 columns]

In [ ]:
finaldf.to_csv("final_data.HepG2.Histone.Enhancer.csv")